In [1]:
with open("../contracts/SimpleStorage.sol", "r") as file:
    simple_storage_file = file.read()

print(simple_storage_file)

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.12;

contract SimpleStorage {

    // this will get initialized to 0
    uint256 favoriteNumber;
    bool favoriteBool;

    struct Person {
        uint256 favoriteNumber;
        string name;
    }

    // this array is dynamic specifying
    Person[] public people;
    // mappings in sol have default values, for uint256 it's 0
    mapping(string => uint256) public nameToFavoriteNumber;

    function store(uint256 _favoriteNumber) public {
        favoriteNumber = _favoriteNumber;
    }

    function retrieve() public view returns (uint256) {
        return favoriteNumber;
    }

    function addPerson(string memory _name, uint256 _favoriteNumber) public {
        people.push(Person(_favoriteNumber, _name));
        nameToFavoriteNumber[_name] = _favoriteNumber;
    }

}

// what if we want to have many instances of this contract?



In [2]:
# now we have to compile our smart contract - will use python solc-x package
from solcx import compile_standard

In [3]:
compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"SimpleStorage.sol": {"content": simple_storage_file}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version="0.8.17",
)


In [4]:
import json

In [5]:
with open("compiled_code.json", "w") as file:
    json.dump(compiled_sol, file)

In [6]:
# get bytecode
bytecode = compiled_sol["contracts"]["SimpleStorage.sol"]["SimpleStorage"]["evm"]["bytecode"]["object"]
# get abi
abi = compiled_sol["contracts"]["SimpleStorage.sol"]["SimpleStorage"]["abi"]

In [7]:
abi

[{'inputs': [{'internalType': 'string', 'name': '_name', 'type': 'string'},
   {'internalType': 'uint256', 'name': '_favoriteNumber', 'type': 'uint256'}],
  'name': 'addPerson',
  'outputs': [],
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'inputs': [{'internalType': 'string', 'name': '', 'type': 'string'}],
  'name': 'nameToFavoriteNumber',
  'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
  'name': 'people',
  'outputs': [{'internalType': 'uint256',
    'name': 'favoriteNumber',
    'type': 'uint256'},
   {'internalType': 'string', 'name': 'name', 'type': 'string'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [],
  'name': 'retrieve',
  'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [{'internalType': 'uint256',


In [9]:
# now deploy

from web3 import Web3

# connect to local ganache
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:7545"))
chain_id = 1337
my_address = "0x59F73B6aEF0E11fcA0BaB47e950674E4Ae693e68"
private_key = "0xcdf6b18fd82ca4bd20f9f456ca375f23c8f731bb3d6972feab9b59d8b280aed6"

In [10]:
SimpleStorage = w3.eth.contract(abi=abi, bytecode=bytecode)
SimpleStorage

web3._utils.datatypes.Contract

In [11]:
# build a transaction that deploys the contract
# get the latest transaction nonce
nonce = w3.eth.getTransactionCount(my_address)
nonce

0

In [13]:
# build a transaction - SimpleStorage does not have a constructor
transaction = SimpleStorage.constructor().buildTransaction({
    "gasPrice": w3.eth.gas_price,
    "chainId": chain_id,
    "from": my_address,
    "nonce": nonce,
})
transaction

{'value': 0,
 'gas': 562541,
 'gasPrice': 20000000000,
 'chainId': 1337,
 'from': '0x59F73B6aEF0E11fcA0BaB47e950674E4Ae693e68',
 'nonce': 0,
 'data': '0x608060405234801561001057600080fd5b5061093b806100206000396000f3fe608060405234801561001057600080fd5b50600436106100575760003560e01c80632e64cec11461005c5780636057361d1461007a5780636f760f41146100965780638bab8dd5146100b25780639e7a13ad146100e2575b600080fd5b610064610113565b60405161007191906102b2565b60405180910390f35b610094600480360381019061008f919061030d565b61011c565b005b6100b060048036038101906100ab9190610480565b610126565b005b6100cc60048036038101906100c791906104dc565b6101af565b6040516100d991906102b2565b60405180910390f35b6100fc60048036038101906100f7919061030d565b6101dd565b60405161010a9291906105a4565b60405180910390f35b60008054905090565b8060008190555050565b6002604051806040016040528083815260200184815250908060018154018082558091505060019003906000526020600020906002020160009091909190915060008201518160000155602082015181600101908161018591906107e0565b505

In [14]:
# sign the transaction
signed_txn = w3.eth.account.sign_transaction(transaction, private_key=private_key)
signed_txn

SignedTransaction(rawTransaction=HexBytes('0xf909b0808504a817c8008308956d8080b9095b608060405234801561001057600080fd5b5061093b806100206000396000f3fe608060405234801561001057600080fd5b50600436106100575760003560e01c80632e64cec11461005c5780636057361d1461007a5780636f760f41146100965780638bab8dd5146100b25780639e7a13ad146100e2575b600080fd5b610064610113565b60405161007191906102b2565b60405180910390f35b610094600480360381019061008f919061030d565b61011c565b005b6100b060048036038101906100ab9190610480565b610126565b005b6100cc60048036038101906100c791906104dc565b6101af565b6040516100d991906102b2565b60405180910390f35b6100fc60048036038101906100f7919061030d565b6101dd565b60405161010a9291906105a4565b60405180910390f35b60008054905090565b8060008190555050565b6002604051806040016040528083815260200184815250908060018154018082558091505060019003906000526020600020906002020160009091909190915060008201518160000155602082015181600101908161018591906107e0565b5050508060038360405161019991906108ee565b908152602001604051809103902081905

In [15]:
# send the transaction
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)

In [16]:
# wait for the transaction to be mined
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
tx_receipt

AttributeDict({'transactionHash': HexBytes('0x55f99fe8d9ec5b2c35bf50c573ea318a56308465f8200d47fe93e606301ad9f2'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x7bc3271f7c866118f01a3cb8162cfd9496a84b16c4e253dae437885268a4d5bc'),
 'blockNumber': 1,
 'from': '0x59F73B6aEF0E11fcA0BaB47e950674E4Ae693e68',
 'to': None,
 'gasUsed': 562541,
 'cumulativeGasUsed': 562541,
 'contractAddress': '0xEd1787112C04A01FF76343aeD1ecc162c2C7c40a',
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [17]:
# working with the contract
# remember we always need the address and abi

simple_storage = w3.eth.contract(address=tx_receipt.contractAddress, abi=abi)

In [18]:
simple_storage.all_functions()

[<Function addPerson(string,uint256)>,
 <Function nameToFavoriteNumber(string)>,
 <Function people(uint256)>,
 <Function retrieve()>,
 <Function store(uint256)>]

In [23]:
# simple_storage.all_functions() returns a list of functions
# [<Function addPerson(string,uint256)>,
#  <Function nameToFavoriteNumber(string)>,
#  <Function people(uint256)>,
#  <Function retrieve()>,
#  <Function store(uint256)>]

# lets call the retrieve function

simple_storage.functions.retrieve().call()

0

In [24]:
# now store a new value

store_txn = simple_storage.functions.store(15).buildTransaction({
    "gasPrice": w3.eth.gas_price,
    "chainId": chain_id,
    "from": my_address,
    "nonce": nonce + 1,
})

store_txn

{'value': 0,
 'gas': 41624,
 'gasPrice': 20000000000,
 'chainId': 1337,
 'from': '0x59F73B6aEF0E11fcA0BaB47e950674E4Ae693e68',
 'nonce': 1,
 'to': '0xEd1787112C04A01FF76343aeD1ecc162c2C7c40a',
 'data': '0x6057361d000000000000000000000000000000000000000000000000000000000000000f'}

In [25]:
# sign the transaction
signed_store_txn = w3.eth.account.sign_transaction(store_txn, private_key=private_key)
signed_store_txn

SignedTransaction(rawTransaction=HexBytes('0xf88a018504a817c80082a29894ed1787112c04a01ff76343aed1ecc162c2c7c40a80a46057361d000000000000000000000000000000000000000000000000000000000000000f820a96a05b9711d947199abf9381eef65edb24f3f8bf177c23f691d42e322e8780f1cd62a04e8af67b14ad6cac1751edee4df580978c8662a5e2918ae564120ea417ac7e4b'), hash=HexBytes('0x819b790247b074c6442e01f194b56e6694ceb64dc13fea7e76557ebde377ccd1'), r=41427386315342805369965738134042747030181080433026475350866814669419638279522, s=35525928232286534403839553234886586074771195382431383334520168789498850475595, v=2710)

In [26]:
# send the transaction
send_store_txn = w3.eth.send_raw_transaction(signed_store_txn.rawTransaction)

# wait for the transaction to be mined
send_store_receipt = w3.eth.wait_for_transaction_receipt(send_store_txn)
send_store_receipt

AttributeDict({'transactionHash': HexBytes('0x819b790247b074c6442e01f194b56e6694ceb64dc13fea7e76557ebde377ccd1'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xd84fc3890e46cce49cb8d7195de87246de82b6111d992be98f2f1e30b640e916'),
 'blockNumber': 2,
 'from': '0x59F73B6aEF0E11fcA0BaB47e950674E4Ae693e68',
 'to': '0xEd1787112C04A01FF76343aeD1ecc162c2C7c40a',
 'gasUsed': 41624,
 'cumulativeGasUsed': 41624,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000')

In [27]:
# now call the retrieve function again

simple_storage.functions.retrieve().call()

15